# 1. 라이브러리 임포트

In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np

import json
import os
from statistics import median

from PIL import Image, ImageOps, ImageDraw
import cv2

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns

from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import ResNet50V2, ResNet101V2, ResNet152V2, ConvNeXtTiny, ConvNeXtSmall, ConvNeXtBase
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, LayerNormalization, Dense

import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
!pip install koreanize-matplotlib
import koreanize_matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 29.1 MB/s eta 0:00:00


In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


# 2. 데이터 불러오기 및 데이터프레임 생성하기

In [ ]:
# json path 생성
label_path = '/content/drive/MyDrive/Data/label'
train_label_path = os.path.join(label_path, 'train')
val_label_path = os.path.join(label_path, 'val')
label_json_list = ['anger.json', 'happy.json', 'panic.json', 'sadness.json']

# image path 생성
image_path = '/content/drive/MyDrive/preprocessed_crop'
train_path = os.path.join(image_path,'train')
val_path = os.path.join(image_path,'val')

# segmentation path 생성
seg_path = '/content/drive/MyDrive/preprocessed_seg'
train_seg_path = os.path.join(seg_path,'train')
val_seg_path = os.path.join(seg_path,'val')

In [ ]:
# train image list 생성
train_path_anger = os.path.join(train_path, 'anger')
train_path_happy = os.path.join(train_path, 'happy')
train_path_panic = os.path.join(train_path, 'panic')
train_path_sadness = os.path.join(train_path, 'sadness')

train_path_anger = os.listdir(train_path_anger)
train_path_happy = os.listdir(train_path_happy)
train_path_panic = os.listdir(train_path_panic)
train_path_sadness = os.listdir(train_path_sadness)

# val image path list 생성
val_path_anger = os.path.join(val_path, 'anger')
val_path_happy = os.path.join(val_path, 'happy')
val_path_panic = os.path.join(val_path, 'panic')
val_path_sadness = os.path.join(val_path, 'sadness')

val_path_anger = os.listdir(val_path_anger)
val_path_happy = os.listdir(val_path_happy)
val_path_panic = os.listdir(val_path_panic)
val_path_sadness = os.listdir(val_path_sadness)

In [ ]:
# train image_seg list 생성
train_seg_path_anger = os.path.join(train_seg_path, 'anger')
train_seg_path_happy = os.path.join(train_seg_path, 'happy')
train_seg_path_panic = os.path.join(train_seg_path, 'panic')
train_seg_path_sadness = os.path.join(train_seg_path, 'sadness')

train_seg_path_anger = os.listdir(train_seg_path_anger)
train_seg_path_happy = os.listdir(train_seg_path_happy)
train_seg_path_panic = os.listdir(train_seg_path_panic)
train_seg_path_sadness = os.listdir(train_seg_path_sadness)

val_seg_path_anger = os.path.join(val_seg_path, 'anger')
val_seg_path_happy = os.path.join(val_seg_path, 'happy')
val_seg_path_panic = os.path.join(val_seg_path, 'panic')
val_seg_path_sadness = os.path.join(val_seg_path, 'sadness')

val_seg_path_anger = os.listdir(val_seg_path_anger)
val_seg_path_happy = os.listdir(val_seg_path_happy)
val_seg_path_panic = os.listdir(val_seg_path_panic)
val_seg_path_sadness = os.listdir(val_seg_path_sadness)

In [ ]:
# image 개수 확인(happy, panic의 개수가 줄어든 이유: 중복된 데이터가 존재했음)
print(len(list(train_path_anger)), len(list(train_path_happy)), len(list(train_path_panic)), len(list(train_path_sadness)))
print(len(list(val_path_anger)), len(list(val_path_happy)), len(list(val_path_panic)), len(list(val_path_sadness)))

# image_seg 개수 확인
print(len(list(train_seg_path_anger)), len(list(train_seg_path_happy)), len(list(train_seg_path_panic)), len(list(train_seg_path_sadness)))
print(len(list(val_seg_path_anger)), len(list(val_seg_path_happy)), len(list(val_seg_path_panic)), len(list(val_seg_path_sadness)))

1500 1494 1500 1500
300 300 300 300
1500 1494 1500 1500
300 300 300 300


In [ ]:
# DataFrame column 생성
train_df = pd.DataFrame(columns=['img_path','age', 'gender', 'isProf', 'background', 'maxX','maxY','minX','minY','label'])
val_df = pd.DataFrame(columns=['img_path','age', 'gender', 'isProf', 'background', 'maxX','maxY','minX','minY','label'])

train_seg_df = pd.DataFrame(columns=['img_path','age', 'gender', 'isProf', 'background', 'maxX','maxY','minX','minY','label'])
val_seg_df = pd.DataFrame(columns=['img_path','age', 'gender', 'isProf', 'background', 'maxX','maxY','minX','minY','label'])

In [ ]:
import json
def makedf(dir_path, df, data_path):
    i = 0
    data_name = dir_path.split('/')[-1]
    for label_name in label_json_list:
        with open(os.path.join(dir_path, data_name + '_' + label_name), 'r', encoding='cp949') as f:
            file = json.load(f)
        for v in file:
            if v['faceExp_uploader'] == '분노':
                label = 'anger'
            elif v['faceExp_uploader'] == '기쁨':
                label = 'happy'
            elif v['faceExp_uploader'] == '당황':
                label = 'panic'
            elif v['faceExp_uploader'] == '슬픔':
                label = 'sadness'
            minX = median([v['annot_A']['boxes']['minX'], v['annot_B']['boxes']['minX'], v['annot_C']['boxes']['minX']])
            minY = median([v['annot_A']['boxes']['minY'], v['annot_B']['boxes']['minY'], v['annot_C']['boxes']['minY']])
            maxX = median([v['annot_A']['boxes']['maxX'], v['annot_B']['boxes']['maxX'], v['annot_C']['boxes']['maxX']])
            maxY = median([v['annot_A']['boxes']['maxY'], v['annot_B']['boxes']['maxY'], v['annot_C']['boxes']['maxY']])
            result_path = os.path.join(data_path, label)
            df.loc[i] = [os.path.join(result_path, v['filename']), v['age'], v['gender'], v['isProf'], v['bg_uploader'], int(maxX), int(maxY), int(minX), int(minY), label]
            i += 1

    return df.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
# DataFrame 생성
train_df = makedf(train_label_path, train_df, train_path)
val_df = makedf(val_label_path, val_df, val_path)

train_seg_df = makedf(train_label_path, train_seg_df, train_seg_path)
val_seg_df = makedf(val_label_path, val_seg_df, val_seg_path)

# 3. Preprocessing

## 3-1. crop + segment된 사진 정보를 이용하여 좌표정보 확인(5개 데이터에서 음수값 확인)

In [ ]:
train_df[(train_df['minX'] < 0) | (train_df['minY'] < 0) | (train_df['maxX'] < 100) | (train_df['maxY'] < 130)]

img_path  age gender isProf  \
316   /content/drive/MyDrive/preprocessed_crop/train...   40      여    일반인   
1027  /content/drive/MyDrive/preprocessed_crop/train...   20      남    일반인   
3837  /content/drive/MyDrive/preprocessed_crop/train...   20      남    일반인   
5123  /content/drive/MyDrive/preprocessed_crop/train...   20      여    일반인   
5270  /content/drive/MyDrive/preprocessed_crop/train...   30      여    일반인   

        background  maxX  maxY  minX  minY    label  
316   공공시설/종교/의료시설   863  1585   -27   462    panic  
1027         도심 환경  1582   709  1001   -81    anger  
3837  공공시설/종교/의료시설   965   857   261   -20    happy  
5123    상업시설/점포/시장  1493   672   870  -204    anger  
5270  공공시설/종교/의료시설   361   386    -2  -102  sadness

## 3-2. 음수값을 0으로 변환

In [ ]:
if (train_df['minX'] < 0).any():
    train_df.loc[train_df['minX'] < 0, 'minX'] = 0

if (train_df['minY'] < 0).any():
    train_df.loc[train_df['minY'] < 0, 'minY'] = 0

In [ ]:
train_df[(train_df['minX'] < 0) | (train_df['minY'] < 0) | (train_df['maxX'] < 100) | (train_df['maxY'] < 130)]

Empty DataFrame
Columns: [img_path, age, gender, isProf, background, maxX, maxY, minX, minY, label]
Index: []

In [ ]:
val_df[(val_df['minX'] < 0) | (val_df['minY'] < 0) | (val_df['maxX'] < 100) | (val_df['maxY'] < 130)]

Empty DataFrame
Columns: [img_path, age, gender, isProf, background, maxX, maxY, minX, minY, label]
Index: []

## 3-3. segmentation 사진 데이터를 이용하여 for 문을 통해 mean값이 10보다 작은 사진 데이터 탐지

이유: 검정색이 대부분인 데이터 즉, 사진 crop이 이상하게 된 데이터를 탐지하여 제거하기 위함

In [ ]:
for i in range(len(train_seg_df)):
    img = cv2.imread(train_seg_df['img_path'][i])
    mean_val1 = np.mean(img, axis=0)
    mean_val2 = np.mean(mean_val1)
    if mean_val2 < 10:
	    print(train_seg_df['img_path'][i], f'index: {i}')

/content/drive/MyDrive/preprocessed/train/happy/k3s935e50e666fee414ce63c6a7cc0563dbee792014b083a74cbcbf602b735rqt.jpg index: 889
/content/drive/MyDrive/preprocessed/train/sadness/vag7dfda5e3b9561fd12c1175d277b1d2c1f22dd2a5d37b3286dd4b3e2d45pili.jpg index: 5270


In [ ]:
for i in range(len(val_seg_df)):
    img = cv2.imread(val_seg_df['img_path'][i])
    mean_val1 = np.mean(img, axis=0)
    mean_val2 = np.mean(mean_val1)
    if mean_val2 < 10:
	    print(val_seg_df['img_path'][i], f'index: {i}')

/content/drive/MyDrive/preprocessed/val/sadness/rb04c532132d836a981bf48f70b3588730632285f2c97b347e1cdb4fa1133mcty.jpg index: 566


## 3-4. 사진 정보 확인 후 drop을 통해 제거

### 3-4-1. train 데이터 제거

In [ ]:
train_df[train_df['img_path'] == '/content/drive/MyDrive/preprocessed_crop/train/happy/k3s935e50e666fee414ce63c6a7cc0563dbee792014b083a74cbcbf602b735rqt.jpg']

img_path  age gender isProf  \
889  /content/drive/MyDrive/preprocessed_crop/train...   40      남    일반인   

    background  maxX  maxY  minX  minY  label  
889    실외 자연환경  1004  1691     0     0  happy

In [ ]:
train_df[train_df['img_path'] == '/content/drive/MyDrive/preprocessed_crop/train/sadness/vag7dfda5e3b9561fd12c1175d277b1d2c1f22dd2a5d37b3286dd4b3e2d45pili.jpg']

img_path  age gender isProf  \
5270  /content/drive/MyDrive/preprocessed_crop/train...   30      여    일반인   

        background  maxX  maxY  minX  minY    label  
5270  공공시설/종교/의료시설   361   386     0     0  sadness

In [ ]:
train_df = train_df.drop(index=[889, 5270])

In [ ]:
train_df[train_df['img_path'] == '/content/drive/MyDrive/preprocessed_crop/train/happy/k3s935e50e666fee414ce63c6a7cc0563dbee792014b083a74cbcbf602b735rqt.jpg']

Empty DataFrame
Columns: [img_path, age, gender, isProf, background, maxX, maxY, minX, minY, label]
Index: []

In [ ]:
train_df[train_df['img_path'] == '/content/drive/MyDrive/preprocessed_crop/train/sadness/vag7dfda5e3b9561fd12c1175d277b1d2c1f22dd2a5d37b3286dd4b3e2d45pili.jpg']

Empty DataFrame
Columns: [img_path, age, gender, isProf, background, maxX, maxY, minX, minY, label]
Index: []

### 3-4-2. val 데이터 제거

In [ ]:
val_df[val_df['img_path'] == '/content/drive/MyDrive/preprocessed_crop/val/sadness/rb04c532132d836a981bf48f70b3588730632285f2c97b347e1cdb4fa1133mcty.jpg']

img_path  age gender isProf  \
566  /content/drive/MyDrive/preprocessed_crop/val/s...   50      여    일반인   

    background  maxX  maxY  minX  minY    label  
566  숙박 및 거주공간  2543  1342  1689   243  sadness

In [ ]:
val_df = val_df.drop(index=[566])

In [ ]:
val_df[val_df['img_path'] == '/content/drive/MyDrive/preprocessed_crop/val/sadness/rb04c532132d836a981bf48f70b3588730632285f2c97b347e1cdb4fa1133mcty.jpg']

Empty DataFrame
Columns: [img_path, age, gender, isProf, background, maxX, maxY, minX, minY, label]
Index: []

## 3-5. 사진 데이터 array 변환, 차원 추가, resize 전처리 작업 수행

In [ ]:
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array

norm_layer = tf.keras.layers.Normalization(
    mean = [0.485 * 255, 0.456 * 255, 0.406 * 255],
    variance = [(0.229 * 255) ** 2, (0.224 * 255) ** 2, (0.225 * 255) ** 2],
)
def preprocess_image(image,size=224):
    image = np.array(image)
    image = tf.expand_dims(image,0)
    image_resized = tf.image.resize(image,(224,224),method='bicubic') # resolution(늘리거나 작게 했을 때 픽셀값을 조정해주는 것)
    return norm_layer(image_resized).numpy()

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_image)
train_gen = train_datagen.flow_from_dataframe( # 연산속도, fitting 면에서 이득 batch_size가 32로 들어감
    dataframe=train_df,
    x_col='img_path',
    y_col='label',
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    target_size=(224,224)
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_image)
val_gen = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='img_path',
    y_col='label',
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    target_size=(224,224)
)

Found 5992 validated image filenames belonging to 4 classes.
Found 1199 validated image filenames belonging to 4 classes.


## 4. Modeling

## 4-1. ResNet

### 4-1-1. ResNet50 V2

In [ ]:
model_resnet50 = ResNet50V2(
    input_shape=(224,224,3),
    include_top=False,
    weights="imagenet",
)
x=GlobalAveragePooling2D()(model_resnet50.output)
outputs=Dense(4, activation='softmax')(x)

resnet50 = Model(inputs=model_resnet50.input, outputs=outputs)

94668760/94668760 [==============================] - 5s 0us/step


In [ ]:
from tensorflow.keras.optimizers import AdamW
resnet50.compile(optimizer=AdamW(learning_rate=1e-4),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau
checkpoint_callback = ModelCheckpoint(
    filepath='model-{epoch:02d}-{val_loss:.2f}.keras',
    monitor="val_accuracy",
    save_best_only=True,
    mode='max'
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-8
)

callbacks = [
    checkpoint_callback,
    reduce_lr
]

In [ ]:
history_resnet50 = resnet50.fit(
    train_gen,
    epochs=10,
    validation_data=val_gen,
    callbacks=callbacks
)

Epoch 1/10
188/188 [==============================] - 4613s 24s/step - loss: 0.6189 - accuracy: 0.7597 - val_loss: 0.6746 - val_accuracy: 0.7656 - lr: 1.0000e-04
Epoch 2/10
188/188 [==============================] - 96s 510ms/step - loss: 0.1979 - accuracy: 0.9331 - val_loss: 0.6624 - val_accuracy: 0.7990 - lr: 1.0000e-04
Epoch 3/10
188/188 [==============================] - 95s 507ms/step - loss: 0.0632 - accuracy: 0.9808 - val_loss: 0.9648 - val_accuracy: 0.7765 - lr: 1.0000e-04
Epoch 4/10
188/188 [==============================] - 93s 497ms/step - loss: 0.0732 - accuracy: 0.9791 - val_loss: 1.2160 - val_accuracy: 0.7323 - lr: 1.0000e-04
Epoch 5/10
188/188 [==============================] - 93s 497ms/step - loss: 0.1017 - accuracy: 0.9645 - val_loss: 0.8388 - val_accuracy: 0.7606 - lr: 1.0000e-04
Epoch 6/10
188/188 [==============================] - 93s 493ms/step - loss: 0.0305 - accuracy: 0.9902 - val_loss: 0.8047 - val_accuracy: 0.7982 - lr: 5.0000e-05
Epoch 7/10
188/188 [========

### 4-1-2. ResNet101 V2

In [ ]:
model_resnet101 = ResNet101V2(
    input_shape=(224,224,3),
    include_top=False,
    weights="imagenet",
)
x = GlobalAveragePooling2D()(model_resnet101.output)
outputs = Dense(4, activation='softmax')(x)

resnet101 = Model(inputs=model_resnet101.input, outputs=outputs)

171317808/171317808 [==============================] - 10s 0us/step


In [ ]:
from tensorflow.keras.optimizers import AdamW
resnet101.compile(optimizer=AdamW(learning_rate=1e-4),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau
checkpoint_callback = ModelCheckpoint(
    filepath='model-{epoch:02d}-{val_loss:.2f}.keras',
    monitor="val_accuracy",
    save_best_only=True,
    mode='max'
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-8
)

callbacks = [
    checkpoint_callback,
    reduce_lr
]

In [ ]:
history_resnet101 = resnet101.fit(
    train_gen,
    epochs=10,
    validation_data=val_gen,
    callbacks=callbacks
)

Epoch 1/10
188/188 [==============================] - 151s 528ms/step - loss: 0.6238 - accuracy: 0.7498 - val_loss: 0.8535 - val_accuracy: 0.7431 - lr: 1.0000e-04
Epoch 2/10
188/188 [==============================] - 99s 528ms/step - loss: 0.2237 - accuracy: 0.9192 - val_loss: 0.7062 - val_accuracy: 0.7798 - lr: 1.0000e-04
Epoch 3/10
188/188 [==============================] - 99s 528ms/step - loss: 0.1014 - accuracy: 0.9665 - val_loss: 0.7690 - val_accuracy: 0.8040 - lr: 1.0000e-04
Epoch 4/10
188/188 [==============================] - 96s 511ms/step - loss: 0.0779 - accuracy: 0.9733 - val_loss: 0.9981 - val_accuracy: 0.7515 - lr: 1.0000e-04
Epoch 5/10
188/188 [==============================] - 95s 506ms/step - loss: 0.1356 - accuracy: 0.9513 - val_loss: 0.8295 - val_accuracy: 0.7615 - lr: 1.0000e-04
Epoch 6/10
188/188 [==============================] - 96s 511ms/step - loss: 0.0519 - accuracy: 0.9826 - val_loss: 0.7957 - val_accuracy: 0.8015 - lr: 5.0000e-05
Epoch 7/10
188/188 [=======

### 4-1-3. ResNet152 V2

In [ ]:
model_resnet152 = ResNet152V2(
    input_shape=(224,224,3),
    include_top=False,
    weights="imagenet",
)
x = GlobalAveragePooling2D()(model_resnet152.output)
outputs = Dense(4, activation='softmax')(x)

resnet152 = Model(inputs=model_resnet152.input, outputs=outputs)

234545216/234545216 [==============================] - 12s 0us/step


In [ ]:
from tensorflow.keras.optimizers import AdamW
resnet152.compile(optimizer=AdamW(learning_rate=1e-4),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau
checkpoint_callback = ModelCheckpoint(
    filepath='model-{epoch:02d}-{val_loss:.2f}.keras',
    monitor="val_accuracy",
    save_best_only=True,
    mode='max'
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-8
)

callbacks = [
    checkpoint_callback,
    reduce_lr
]

In [ ]:
history_resnet152 = resnet152.fit(
    train_gen,
    epochs=10,
    validation_data=val_gen,
    callbacks=callbacks
)

Epoch 1/10
188/188 [==============================] - 188s 565ms/step - loss: 0.6022 - accuracy: 0.7695 - val_loss: 0.6970 - val_accuracy: 0.7690 - lr: 1.0000e-04
Epoch 2/10
188/188 [==============================] - 104s 554ms/step - loss: 0.2327 - accuracy: 0.9189 - val_loss: 0.7356 - val_accuracy: 0.7773 - lr: 1.0000e-04
Epoch 3/10
188/188 [==============================] - 99s 526ms/step - loss: 0.1291 - accuracy: 0.9544 - val_loss: 0.8475 - val_accuracy: 0.7523 - lr: 1.0000e-04
Epoch 4/10
188/188 [==============================] - 99s 524ms/step - loss: 0.0845 - accuracy: 0.9695 - val_loss: 0.8515 - val_accuracy: 0.7548 - lr: 1.0000e-04
Epoch 5/10
188/188 [==============================] - 103s 549ms/step - loss: 0.0375 - accuracy: 0.9880 - val_loss: 0.8148 - val_accuracy: 0.8123 - lr: 5.0000e-05
Epoch 6/10
188/188 [==============================] - 104s 555ms/step - loss: 0.0090 - accuracy: 0.9983 - val_loss: 0.8136 - val_accuracy: 0.8224 - lr: 5.0000e-05
Epoch 7/10
188/188 [====

## 4-2. ConvNeXt

### 4-2-1.ConvNeXtTiny

In [ ]:
model_convnext_tiny = ConvNeXtTiny(
    input_shape=(224,224,3),
    include_top=False,
    weights="imagenet",
)
x = model_convnext_tiny.layers[-2].output
x = GlobalAveragePooling2D()(x)
x = LayerNormalization()(x)
outputs = Dense(4, activation='softmax')(x)

convnext_tiny = Model(inputs=model_convnext_tiny.input, outputs=outputs)

111650432/111650432 [==============================] - 7s 0us/step


In [ ]:
  from tensorflow.keras.optimizers import AdamW
  convnext_tiny.compile(optimizer=AdamW(learning_rate=1e-4),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau
checkpoint_callback = ModelCheckpoint(
    filepath='model-{epoch:02d}-{val_loss:.2f}.keras',
    monitor="val_accuracy",
    save_best_only=True,
    mode='max'
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-8
)

callbacks = [
    checkpoint_callback,
    reduce_lr
]

In [ ]:
history_convnext_tiny = convnext_tiny.fit(
    train_gen,
    epochs=10,
    validation_data=val_gen,
    callbacks=callbacks
)

Epoch 1/10
188/188 [==============================] - 152s 620ms/step - loss: 0.8881 - accuracy: 0.6185 - val_loss: 0.5949 - val_accuracy: 0.7840 - lr: 1.0000e-04
Epoch 2/10
188/188 [==============================] - 110s 587ms/step - loss: 0.4391 - accuracy: 0.8313 - val_loss: 0.5108 - val_accuracy: 0.8173 - lr: 1.0000e-04
Epoch 3/10
188/188 [==============================] - 108s 574ms/step - loss: 0.2994 - accuracy: 0.8907 - val_loss: 0.5783 - val_accuracy: 0.8015 - lr: 1.0000e-04
Epoch 4/10
188/188 [==============================] - 107s 570ms/step - loss: 0.1866 - accuracy: 0.9366 - val_loss: 0.6618 - val_accuracy: 0.7998 - lr: 1.0000e-04
Epoch 5/10
188/188 [==============================] - 109s 578ms/step - loss: 0.1058 - accuracy: 0.9638 - val_loss: 0.7027 - val_accuracy: 0.8165 - lr: 1.0000e-04
Epoch 6/10
188/188 [==============================] - 109s 581ms/step - loss: 0.0219 - accuracy: 0.9952 - val_loss: 0.7473 - val_accuracy: 0.8232 - lr: 5.0000e-05
Epoch 7/10
188/188 [==

### 4-2-2. ConvNeXtSmall

In [ ]:
model_convnext_small = ConvNeXtSmall(
    input_shape=(224,224,3),
    include_top=False,
    weights="imagenet",
)
x = model_convnext_small.layers[-2].output
x = GlobalAveragePooling2D()(x)
x = LayerNormalization()(x)
outputs = Dense(4, activation='softmax')(x)

convnext_small = Model(inputs=model_convnext_small.input, outputs=outputs)

In [ ]:
from tensorflow.keras.optimizers import AdamW
convnext_small.compile(optimizer=AdamW(learning_rate=1e-4),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau
checkpoint_callback = ModelCheckpoint(
    filepath='model-{epoch:02d}-{val_loss:.2f}.keras',
    monitor="val_accuracy",
    save_best_only=True,
    mode='max'
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-8
)

callbacks = [
    checkpoint_callback,
    reduce_lr
]

In [ ]:
history_convnext_small = convnext_small.fit(
    train_gen,
    epochs=10,
    validation_data=val_gen,
    callbacks=callbacks
)

Epoch 1/10
188/188 [==============================] - 205s 748ms/step - loss: 1.4179 - accuracy: 0.2467 - val_loss: 1.3977 - val_accuracy: 0.2502 - lr: 1.0000e-04
Epoch 2/10
188/188 [==============================] - 130s 693ms/step - loss: 0.9293 - accuracy: 0.5880 - val_loss: 0.5586 - val_accuracy: 0.7873 - lr: 1.0000e-04
Epoch 3/10
188/188 [==============================] - 130s 693ms/step - loss: 0.4187 - accuracy: 0.8390 - val_loss: 0.5682 - val_accuracy: 0.8073 - lr: 1.0000e-04
Epoch 4/10
188/188 [==============================] - 127s 673ms/step - loss: 0.2511 - accuracy: 0.9067 - val_loss: 0.6065 - val_accuracy: 0.7923 - lr: 1.0000e-04
Epoch 5/10
188/188 [==============================] - 123s 654ms/step - loss: 0.1279 - accuracy: 0.9558 - val_loss: 0.7160 - val_accuracy: 0.7973 - lr: 1.0000e-04
Epoch 6/10
188/188 [==============================] - 127s 674ms/step - loss: 0.0265 - accuracy: 0.9933 - val_loss: 0.8967 - val_accuracy: 0.8023 - lr: 5.0000e-05
Epoch 7/10
188/188 [==

### 4-2-3. ConvNeXtBase

In [ ]:
model_convnext_base = ConvNeXtBase(
    input_shape=(224,224,3),
    include_top=False,
    weights="imagenet",
)
x = model_convnext_base.layers[-2].output
x = GlobalAveragePooling2D()(x)
x = LayerNormalization()(x)
outputs = Dense(4, activation='softmax')(x)

convnext_base = Model(inputs=model_convnext_base.input, outputs=outputs)

In [ ]:
from tensorflow.keras.optimizers import AdamW
convnext_base.compile(optimizer=AdamW(learning_rate=1e-4),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau
checkpoint_callback = ModelCheckpoint(
    filepath='model-{epoch:02d}-{val_loss:.2f}.keras',
    monitor="val_accuracy",
    save_best_only=True,
    mode='max'
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-8
)

callbacks = [
    checkpoint_callback,
    reduce_lr
]

In [ ]:
history_convnext_base = convnext_base.fit(
    train_gen,
    epochs=10,
    validation_data=val_gen,
    callbacks=callbacks
)

Epoch 1/10
188/188 [==============================] - 224s 840ms/step - loss: 1.4360 - accuracy: 0.2532 - val_loss: 1.3885 - val_accuracy: 0.2502 - lr: 1.0000e-04
Epoch 2/10
188/188 [==============================] - 142s 749ms/step - loss: 1.4050 - accuracy: 0.2543 - val_loss: 1.3860 - val_accuracy: 0.2502 - lr: 1.0000e-04
Epoch 3/10
188/188 [==============================] - 144s 765ms/step - loss: 1.3993 - accuracy: 0.2632 - val_loss: 1.3458 - val_accuracy: 0.2977 - lr: 1.0000e-04
Epoch 4/10
188/188 [==============================] - 148s 785ms/step - loss: 0.9736 - accuracy: 0.5591 - val_loss: 0.7823 - val_accuracy: 0.6764 - lr: 1.0000e-04
Epoch 5/10
188/188 [==============================] - 146s 777ms/step - loss: 0.5303 - accuracy: 0.7947 - val_loss: 0.5698 - val_accuracy: 0.7940 - lr: 1.0000e-04
Epoch 6/10
188/188 [==============================] - 146s 774ms/step - loss: 0.3826 - accuracy: 0.8571 - val_loss: 0.5650 - val_accuracy: 0.8065 - lr: 1.0000e-04
Epoch 7/10
188/188 [==

# 5. Save Model

In [ ]:
resnet50.save('/content/drive/MyDrive/resnet50.h5')

In [ ]:
resnet101.save('/content/drive/MyDrive/resnet101.h5')

In [ ]:
resnet152.save('/content/drive/MyDrive/resnet152.h5')

In [ ]:
convnext_tiny.save('/content/drive/MyDrive/convnext_tiny.h5')

In [ ]:
convnext_small.save('/content/drive/MyDrive/convnext_small.h5')

In [ ]:
convnext_base.save('/content/drive/MyDrive/convnext_base.h5')